# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235946716512                   -0.50    8.0         
  2   -7.250427702452       -1.84       -1.39    1.0   12.1ms
  3   -7.251200850878       -3.11       -2.24    1.0   12.1ms
  4   -7.251315253752       -3.94       -2.30    2.0   14.2ms
  5   -7.251337888019       -4.65       -2.91    1.0   12.0ms
  6   -7.251338557058       -6.17       -3.36    2.0   14.3ms
  7   -7.251338594877       -7.42       -3.50    1.0   12.7ms
  8   -7.251338776551       -6.74       -3.94    1.0   12.4ms
  9   -7.251338798038       -7.67       -4.59    2.0   13.9ms
 10   -7.251338798592       -9.26       -5.17    2.0   14.6ms
 11   -7.251338798697       -9.98       -5.50    2.0   15.2ms
 12   -7.251338798703      -11.18       -6.12    1.0   13.2ms
 13   -7.251338798704      -12.19       -6.23    2.0   14.9ms
 14   -7.251338798705      -12.40       -6.86    1.0   75.3ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06220195209883573
[ Info: Arnoldi iteration step 2: normres = 0.3359524533235789
[ Info: Arnoldi iteration step 3: normres = 1.0175861719013046
[ Info: Arnoldi iteration step 4: normres = 0.30966363081624626
[ Info: Arnoldi iteration step 5: normres = 0.32814785667405755
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.07e-02, 2.80e-02, 2.47e-01, 2.01e-01, 6.98e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3489717203095167
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.02e-02, 2.89e-01, 1.70e-01, 7.94e-02, 1.55e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09820359213710973
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.01e-04, 2.58e-02, 5.51e-03, 3.25e-02, 8.39e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10532948167707908
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.26e